In [1]:
#Bilder herunterladen

In [15]:
import pandas as pd
import urllib
from skimage.metrics import structural_similarity as compare_ssim
import argparse
import imutils
import cv2
import os

df = pd.read_csv('panini.csv', dtype="unicode")
df = df.fillna("")

df_neu = df[df["images_source"].str.contains("http://laststicker.com/i/cards")]

df_geloescht = df_neu[~df_neu[["images_id", "images_source"]].duplicated()]

df2 = df_geloescht[~df_geloescht["items_name"].str.contains(
    "Team|Stadium|Logo|Emblem|Poster|FIFA|Mascot|Official posters|Badge|Trophy|Intro|Flag|/|adidas")]

df3 = df2[~df2["section"].str.contains(
    "FIFA|Official posters|Federations emblems|Checklist|Team|Stade|Stadiums|Introduction|Stadium|History|RARE / Invincible|Invincible|Group|Host cities' posters|International update|Venues|Extra stickers|Passion & Pride|Roll|Estadio|Hosts")]

#df4 = df3[~df3["collections_id"].str.contains("29|34|35|77|100")]
df4 = df3[df3["collections_id"].str.contains("25")]
df4["items_name"]

df_collection = df_geloescht[~df_geloescht["items_name"].str.contains("Team|Stadium|Logo")]["collections_id"].unique()
df_collection

df5 = df4["items_name"].unique()[:200]
df5

i = 0

for row in df5:
    image_url = df4["images_source"].values[i]
    if os.path.isfile('Spieler/' + df4["items_name"].values[i] + ".jpg") == False:
        save_name = 'Spieler/' + df4["items_name"].values[i] + ".jpg"  #local name to be saved
        urllib.request.urlretrieve(image_url, save_name)
    i += 1



In [ ]:
#Erkennen des Spielers

In [12]:
import numpy
import requests

dim = (60, 60)
img = cv2.imread("bild.jpg")
img2 = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
grayA = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
j = 0
erkannt = ""

for row in df5:
    spieler = str(j) + " Spieler/" + df5[j] + ".jpg"
    #print(spieler)
    stream = open("Spieler/" + df5[j] + ".jpg", "rb")
    bytes = bytearray(stream.read())
    numpyarray = numpy.asarray(bytes, dtype=numpy.uint8)
    bgrImage = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
    dim = (60, 60)
    bild2 = cv2.resize(bgrImage, dim, interpolation=cv2.INTER_AREA)
    grayB = cv2.cvtColor(bild2, cv2.COLOR_BGR2GRAY)
    (score, diff) = compare_ssim(grayA, grayB, full=True)
    diff = (diff * 255).astype("uint8")

    if (score == 1.0):
        erkannt = df5[j]
        break

    j += 1

'Jordi Alba'

In [ ]:
#Anbindung an API

In [13]:
request = "https://soccer.sportmonks.com/api/v2.0/players/search/" + erkannt + "?api_token=U78MwhoR0QAN1B0qEpry96PQKpep8wUYbUmlCmRnRFXxu98KiEXzwlojj85h"
response2 = requests.get(request)
response2.json()["data"]

#Aufteilen in Vor& Nachname
namen = erkannt.split()
namen[-1]

match = []
match1 = []

for i in response2.json()["data"]:
    if i[
        "display_name"] == erkannt:  #Wenn es exakt passt, dann wir das gleich übernommen. Sonst wird es auf Vor- und Nachname aufgesplittet
        match = i
    elif i["lastname"].startswith(namen[-1]):  #Check Nachname
        match1.append(i)  #Liste mit passenden Nachnamen
        if i["firstname"].startswith(namen[0]):
            match.append(i)

if len(match) == 0:
    for i in match1:
        if i["firstname"].startswith(namen[0][0]):
            match.append(i)

print(match["common_name"])
print(match["birthdate"])
match

J. Alba Ramos
21/03/1989


{'player_id': 185530,
 'team_id': 83,
 'country_id': 32,
 'position_id': 2,
 'common_name': 'J. Alba Ramos',
 'display_name': 'Jordi Alba',
 'fullname': 'Jordi Alba Ramos',
 'firstname': 'Jordi',
 'lastname': 'Alba Ramos',
 'nationality': 'Spain',
 'birthdate': '21/03/1989',
 'birthcountry': 'Spain',
 'birthplace': "L'Hospitalet de Llobregat",
 'height': '170 cm',
 'weight': '68 kg',
 'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/26/185530.png'}